# 数据预处理实战

本教程将带你完成一个完整的数据预处理流程。我们将使用真实的数据集进行演示。

## 环境准备

首先导入必要的库：

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 1. 数据加载与查看

我们使用一个房价预测数据集作为示例。这个数据集包含了房屋的各种特征（如面积、建造年份等）和销售价格。

通过以下步骤我们将：
- 加载数据集
- 查看基本信息
- 了解数据分布

In [ ]:
# 加载数据
df = pd.read_csv('house_prices.csv')

# 查看基本信息
print("数据集基本信息：")
print(df.info())

print("\n数据预览：")
display(df.head())

print("\n数值型特征统计：")
display(df.describe())

## 2. 缺失值处理

在实际数据中，缺失值是常见的问题。我们需要：
- 检测缺失值的分布
- 分析缺失的原因
- 选择合适的填充策略

下面我们将检查并处理数据集中的缺失值：

In [ ]:
# 查看缺失值情况
missing = df.isnull().sum()
missing_percent = missing / len(df) * 100

missing_df = pd.DataFrame({
    '缺失值数量': missing,
    '缺失比例': missing_percent
}).query('缺失值数量 > 0')

print("缺失值统计：")
display(missing_df)

# 处理缺失值
# 1. 数值型特征用中位数填充
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# 2. 类别型特征用众数填充
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

## 3. 异常值检测

异常值可能会显著影响模型性能。我们使用箱线图来可视化检测异常值：
- 箱线图可以直观显示数据的分布情况
- 可以快速识别潜在的异常点
- 帮助我们理解数据的范围和集中趋势

In [ ]:
def plot_boxplot(df, features, figsize=(12, 6)):
    plt.figure(figsize=figsize)
    sns.boxplot(data=df[features])
    plt.xticks(rotation=45)
    plt.title('数值特征箱线图')
    plt.show()

# 选择主要的数值特征绘制箱线图
main_features = ['LotArea', 'YearBuilt', 'TotalBsmtSF', 'GrLivArea', 'SalePrice']
plot_boxplot(df, main_features)

## 4. 特征工程

特征工程是提高模型性能的关键步骤。我们将：
- 创建新的特征
- 对类别特征进行编码
- 对数值特征进行标准化

这些步骤可以帮助我们：
- 提取更多有用信息
- 将特征转换为模型可用的格式
- 消除不同特征之间的尺度差异

In [ ]:
# 1. 创建新特征
df['House_Age'] = 2024 - df['YearBuilt']
df['Total_SF'] = df['TotalBsmtSF'] + df['GrLivArea']

# 2. 类别特征编码
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in categorical_cols:
    df[col + '_Encoded'] = le.fit_transform(df[col])

# 3. 数值特征标准化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[numeric_cols + '_Scaled'] = scaler.fit_transform(df[numeric_cols])

## 5. 特征选择

并非所有特征都对预测有帮助。通过特征选择，我们可以：
- 识别重要特征
- 降低数据维度
- 减少噪声影响
- 提高模型效率

下面我们使用相关性分析来选择重要特征：

In [ ]:
# 计算相关性矩阵
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
correlation_matrix = df[numeric_features].corr()

# 绘制热力图
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt='.2f')
plt.title('特征相关性热力图')
plt.show()

## 总结

在这个实践中，我们完成了：
1. 数据加载和基本探索
2. 缺失值的检测和处理
3. 异常值的识别
4. 特征工程和转换
5. 特征重要性分析

这些是数据预处理的基本步骤，为后续的建模打下了基础。

### 练习建议
- 尝试使用不同的填充方法
- 探索其他特征工程技巧
- 思考如何处理极端异常值
- 尝试其他特征选择方法